In [77]:
# import movie metadata into pandas Dataframes
import pandas as pd
import re
kaggle_metadata = pd.read_csv('Data/movies_metadata.csv', low_memory=False)
ratings = pd.read_csv('Data/ratings.csv')
wiki_movies_df = pd.read_csv('Data/wikidata.csv')

In [78]:
# Examine our extracted data --------------------------------
# Instead of using head() and tail() methods, we can use the sample(n=<# of values to pull>) method to randomly pull a number of values from the dataframe
# Pull 10 values from the kaggle_metadata dataframe
kaggle_metadata.sample(n=10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
15431,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,51617,tt0037367,en,This Happy Breed,A middle-class family faces personal triumphs ...,...,1944-06-01,0.00,115.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,This Happy Breed,False,6.60,11.00
5536,False,NaN,25000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,8051,tt0272338,en,Punch-Drunk Love,A beleaguered small-business owner gets a harm...,...,2002-10-25,"17,000,000.00",95.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Punch-Drunk Love,False,6.90,542.00
21041,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,42714,tt0060112,en,Any Wednesday,"Ellen Gordon, a New York executive's mistress ...",...,1966-10-13,0.00,109.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It's got some new ideas about multiple dwelling!,Any Wednesday,False,6.10,5.00
38056,False,NaN,0,[],NaN,191427,tt0002985,en,The House of Darkness,No Overview,...,1913-05-10,0.00,17.00,[],Released,NaN,The House of Darkness,False,4.30,3.00
26835,False,NaN,0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 878, 'n...",NaN,16356,tt0310952,en,Riverworld,A movie for the Sci Fi Channel based on the bo...,...,2003-03-22,0.00,86.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Riverworld,False,4.40,10.00
11860,False,NaN,0,"[{'id': 9648, 'name': 'Mystery'}, {'id': 80, '...",http://www.paranoidpark.co.uk/,1990,tt0842929,en,Paranoid Park,The teenager and skateboarder Alex is intervie...,...,2007-05-21,"486,021.00",85.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Could you hide a deadly secret?,Paranoid Park,False,6.30,126.00
28215,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,320003,tt2711898,fr,Journal d'une femme de chambre,"France, around 1900. Coming from the vibrancy ...",...,2015-02-07,0.00,96.00,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,Diary of a Chambermaid,False,5.20,41.00
1157,False,NaN,15000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 878, 'na...",NaN,68,tt0088846,en,Brazil,Low-level bureaucrat Sam Lowry escapes the mon...,...,1985-02-20,0.00,132.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It's only a state of mind.,Brazil,False,7.60,877.00
39330,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",http://www.dragbecomeshim.com/,338063,tt3593124,en,Drag Becomes Him,Drag Becomes Him provides an intimate glimpse ...,...,2015-04-29,0.00,82.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Drag Becomes Him,False,8.00,1.00
15601,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",NaN,46011,tt0045592,en,Call Me Madam,Washington hostess Sally Adams becomes a Truma...,...,1953-03-25,0.00,114.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Call Me Madam,False,8.50,2.00


In [79]:
# Convert output of read_csv to DataFrame
kaggle_metadata = pd.DataFrame(kaggle_metadata)

In [80]:
# Cast the dataframe's columns to a list to inspect
kaggle_metadata.columns.tolist()

['adult',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count']

In [81]:
# Planning our ETL pipeline
# 1. Inspect
# 2. Plan
# 3. Execute

# We want to slim down the number of columns in the above dataframe to a more manageable and RELEVANT number

In [82]:
kaggle_metadata.dtypes

adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object

In [83]:
# Convert discrepant data types:
# video: Boolean
# budget: numeric
# id: numeric
# release_date: datetime
# popularity: numeric
# adult: Boolean


In [84]:
# Tackle Adult column:

# Check data
kaggle_metadata['adult'].value_counts()

False                                                                                                                             45454
True                                                                                                                                  9
 - Written by Ørnås                                                                                                                   1
 Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.        1
 Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.                        1
Name: adult, dtype: int64

In [85]:
# Clean data
# The following uses Pandas boolean operator ~ (aka all values that are False for .isin(['True','False']), or values that aren't True or False)
kaggle_metadata[~kaggle_metadata['adult'].isin(['True','False'])]


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,nan,nan,NaN,NaN,NaN,NaN,NaN,nan,nan
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,nan,nan,NaN,NaN,NaN,NaN,NaN,nan,nan
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,nan,nan,NaN,NaN,NaN,NaN,NaN,nan,nan


In [86]:
# Since we don't want to include Adult movies in our HACKATHON dataset, we should keep rows where Adult is False, and drop all True rows
kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult',axis='columns')


In [87]:
# Check values of the video column
kaggle_metadata['video'].value_counts()


False    45358
True        93
Name: video, dtype: int64

In [88]:
# This data is clean!
# Since we're using the == Boolean comparator, we can automatically convert this string-Boolean column into a Boolean column
# All values that equal the string 'True' will return the Boolean True
# All else will be false
# Convert, and reassign this to the 'video' column
kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'

In [89]:
# Convert remaining data types

# Convert budget to integer
kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
# Using to_numeric() method ['id']; if data cannot be converted to a number, use the errors='raise' argument to raise an error
kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')

In [90]:
# Convert release_date to datetime
kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])


In [91]:
# Reasonability checks on ratings data -------------------
# Examine the ratings data
# .info() method returns summary data on a dataframe
ratings.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
 #   Column     Non-Null Count     Dtype  
---  ------     --------------     -----  
 0   userId     26024289 non-null  int64  
 1   movieId    26024289 non-null  int64  
 2   rating     26024289 non-null  float64
 3   timestamp  26024289 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 794.2 MB


In [92]:
# Convert timestamp to the datetime format, specifically unix (seconds since epoch)
# use the pd.to_dateimte() method

pd.to_datetime(ratings['timestamp'], unit='s')

# Examine the data

0          2015-03-09 22:52:09
1          2015-03-09 23:07:15
2          2015-03-09 22:52:03
3          2015-03-09 22:52:26
4          2015-03-09 22:52:36
                   ...        
26024284   2009-10-31 23:26:04
26024285   2009-10-31 23:33:52
26024286   2009-10-31 23:29:24
26024287   2009-11-01 00:06:30
26024288   2009-10-31 23:30:58
Name: timestamp, Length: 26024289, dtype: datetime64[ns]

In [93]:
# Reassign the newly formatted timestamp column
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')


In [94]:
# Examine the rating data for outliers and dirty data using the .describe() method, following by reporting to a histogram

# Format our description output to 2 decimals
pd.options.display.float_format = '{:20,.2f}'.format
# Plot our Series into a histogram
ratings['rating'].plot(kind='hist')
# Print descriptive statistics
ratings['rating'].describe()

count          26,024,289.00
mean                    3.53
std                     1.07
min                     0.50
25%                     3.00
50%                     3.50
75%                     4.00
max                     5.00
Name: rating, dtype: float64

In [95]:
# Combine Wikipedia and Kaggle datasets ------------

# Since both data share the imdb_id column, we will use this as a PRIMARY KEY and perform an INNER JOIN (merge) on this
# Check for redundant columns; by using the suffixes=[] argument, we can append suffixes to the columns taken from both datasets that share a name
movies_df = pd.merge(wiki_movies_df, kaggle_metadata, on='imdb_id', suffixes=['_wiki','_kaggle'])




In [96]:
# Check which columns are duplicated in both datasets
movies_df.columns.to_list()

['Unnamed: 0',
 'url',
 'year',
 'imdb_link',
 'title_wiki',
 'Based on',
 'Starring',
 'Cinematography',
 'Running time',
 'Country',
 'Language',
 'Director',
 'Distributor',
 'Editor(s)',
 'Composer(s)',
 'Producer(s)',
 'Production company(s)',
 'Writer(s)',
 'imdb_id',
 'box_office',
 'budget_wiki',
 'release_date_wiki',
 'running_time',
 'belongs_to_collection',
 'budget_kaggle',
 'genres',
 'homepage',
 'id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date_kaggle',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title_kaggle',
 'video',
 'vote_average',
 'vote_count']

In [97]:
# Documenting data merging resolutions
# This is important for sharing your analysis -if we make decisions re: dropping data etc, we should have this documented clearly

# Competing data:
# Wiki                     Movielens                Resolution
#--------------------------------------------------------------------------
# title_wiki               title_kaggle             Drop Wiki
# running_time             runtime                  Keep Kaggle, replace 0 values with Wiki
# budget_wiki              budget_kaggle            Keep Kaggle, replace 0 values with Wiki
# box_office               revenue                  Keep Kaggle, replace 0 values with Wiki
# release_date_wiki        release_date_kaggle      Drop Wiki
# Language                 original_language        Drop Wiki
# Production company(s)    production_companies     Drop Wiki

In [98]:
# Inspect both title columns to see how the data aligns
movies_df[['title_wiki','title_kaggle']]


,title_wiki,title_kaggle
0,The Adventures of Ford Fairlane,The Adventures of Ford Fairlane
1,"After Dark, My Sweet","After Dark, My Sweet"
2,Air America,Air America
3,Alice,Alice
4,Almost an Angel,Almost an Angel
...,...,...
6072,A Fantastic Woman,A Fantastic Woman
6073,Permission,Permission
6074,Loveless,Loveless
6075,Gemini,Gemini


In [99]:
# Return only rows in which both title columns do not match
# Note the first index set returns only the rows where values do not match; the second [] index returns only the title columns from the newly created dataframe
movies_df[movies_df['title_wiki'] != movies_df['title_kaggle']][['title_wiki','title_kaggle']]


,title_wiki,title_kaggle
27,China Cry,China Cry: A True Story
36,Daddy's Dyin' ...Who's Got the Will?,Daddy's Dyin'... Who's Got the Will?
38,NaN,Delusion
39,NaN,Show
40,NaN,How I Unleashed World War II Part III: Among F...
...,...,...
5981,Chips,CHiPS
5996,Spark,Spark: A Space Tail
6019,Pirates of the Caribbean:Dead Men Tell No Tales,Pirates of the Caribbean: Dead Men Tell No Tales
6048,Valerian and the Cityof a Thousand Planets,Valerian and the City of a Thousand Planets


In [100]:
# Show any rows where title_kaggle is empty
movies_df[(movies_df['title_kaggle'] == '') | (movies_df['title_kaggle'].isnull())]

,Unnamed: 0,url,year,imdb_link,title_wiki,Based on,Starring,Cinematography,Running time,Country,...,release_date_kaggle,revenue,runtime,spoken_languages,status,tagline,title_kaggle,video,vote_average,vote_count


In [101]:
# Since there are no rows that title_kaggle is empty, we can assume the Kaggle data is higher resolution than wikipedia data and subsequently drop the title_wiki column

# DECISION & DOCUMENT: Drop Wiki data
movies_df.drop('title_wiki', axis=1, inplace=True)

In [102]:
# Runtime vs running_time

# If we create a scatter plot of runtime vs running_time, we can expect that, if the columns were equal, we would have a perfectly straight line where y = x
# Deviations from this straight line indicate missing or discordant data in one of the datasets

# We fill NaN values with 0 and plot our columns
movies_df.fillna(0).plot(x='running_time', y='runtime', kind='scatter')

# DECISION & DOCUMENT: Keep Kaggle data, replace missing values with Wiki data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='running_time', ylabel='runtime'>

In [103]:
# budget_wiki and budget_kaggle

# Create another scatter plot to compare values
movies_df.fillna(0).plot(x='budget_wiki',y='budget_kaggle', kind='scatter')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='budget_wiki', ylabel='budget_kaggle'>

In [104]:
# Questions to consider:
'''
Which dataset seems to have more outliers?
Which dataset seems to have more missing data points?
If we were to fill in the missing data points of one set with the other, which would be more likely to give us consistent data?
Is it better to start with a base of consistent data and fill in missing points with possible outliers? Or is it better to start with a base of data with outliers and fill in missing points with more consistent data?
'''
# DECISION & DOCUMENT: Wikipedia data has more outliers and Kaggle data looks more complete; we can use Kaggle data and fill 0s with Wikipedia data


'\nWhich dataset seems to have more outliers?\nWhich dataset seems to have more missing data points?\nIf we were to fill in the missing data points of one set with the other, which would be more likely to give us consistent data?\nIs it better to start with a base of consistent data and fill in missing points with possible outliers? Or is it better to start with a base of data with outliers and fill in missing points with more consistent data?\n'

In [105]:
# box_office and revenue
# Use another scatter plot

movies_df.fillna(0).plot(x='box_office', y='revenue', kind='scatter')



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='box_office', ylabel='revenue'>

In [106]:
# Graph is protracted due to the large >2.5 bil datapoint
# Replace NaN values with 0 and return values less than 1 billion to increase resolution of majority of datapoints
movies_df.fillna(0)[movies_df['box_office'] < 10**9].plot(x='box_office', y='revenue', kind='scatter')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='box_office', ylabel='revenue'>

In [107]:
# DECISION & DOCUMENT: Wikipedia looks like it has more outliers, so use Kaggle data and fill 0 values with Wikipedia data

In [108]:
# release_date_wiki and release_date_kaggle

# Since release date time is in datetime format, so we can't use a scatter plot to plot these
# Instead, a line plot can be converted to use dots as the style using style='.' with the plot() method

movies_df[['release_date_wiki','release_date_kaggle']].plot(x='release_date_wiki', y='release_date_kaggle', style='.')




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='release_date_wiki'>

In [109]:
# Investigate outliers

movies_df[(movies_df['release_date_wiki'] > '1996-01-01') & (movies_df['release_date_kaggle'] < '1965-01-01')]




,Unnamed: 0,url,year,imdb_link,Based on,Starring,Cinematography,Running time,Country,Language,...,release_date_kaggle,revenue,runtime,spoken_languages,status,tagline,title_kaggle,video,vote_average,vote_count
3631,4246,https://en.wikipedia.org/wiki/The_Holiday,"2,006.00",https://www.imdb.com/title/tt00457939/,NaN,"['Kate Winslet', 'Cameron Diaz', 'Jude Law', '...",Dean Cundey,136 minutes,United States,English,...,1953-08-28,"30,500,000.00",118.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Pouring out of impassioned pages...brawling th...,From Here to Eternity,False,7.20,137.00


In [110]:
# The above row got accidentally merged (from two movies to one). We can drop this row by accessing the index property of the row (.index)
movies_df = movies_df.drop(movies_df[(movies_df['release_date_wiki'] > '1996-01-01') & (movies_df['release_date_kaggle'] < '1965-01-01')].index)

# Compare number of missing rows from Kaggle release_date column and Wiki column
movies_df[movies_df['release_date_kaggle'].isnull()]



,Unnamed: 0,url,year,imdb_link,Based on,Starring,Cinematography,Running time,Country,Language,...,release_date_kaggle,revenue,runtime,spoken_languages,status,tagline,title_kaggle,video,vote_average,vote_count
41,46,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,0.00,110.00,[],Released,Fear is the greatest killer.,Dreamkiller,False,5.00,1.00
44,46,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,0.00,98.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Rookie detective Endeavor Morse faces his firs...,Endeavour,False,6.60,19.00
54,46,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,0.00,0.00,[],Released,NaN,The Garden of Afflictions 2017,False,0.00,0.00


In [132]:
# Check wiki data
movies_df[movies_df['release_date_wiki'].isnull()]


,Unnamed: 0,url,year,imdb_link,Based on,Starring,Cinematography,Running time,Country,Language,...,release_date_kaggle,revenue,runtime,spoken_languages,status,tagline,title_kaggle,video,vote_average,vote_count
38,46,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1991-06-07,0.00,100.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,She only wanted love. But money's better than ...,Delusion,False,4.80,3.00
39,46,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2003-03-19,0.00,nan,[],Released,NaN,Show,False,6.30,2.00
40,46,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1970-04-06,0.00,73.00,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Released,NaN,How I Unleashed World War II Part III: Among F...,False,7.00,3.00
41,46,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,0.00,110.00,[],Released,Fear is the greatest killer.,Dreamkiller,False,5.00,1.00
42,46,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1979-03-09,0.00,78.00,[],Rumored,NaN,The Winner,False,0.00,0.00
43,46,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2005-11-05,0.00,87.00,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Before The Dinosaurs - Walking With Monsters,False,7.30,12.00
44,46,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,0.00,98.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Rookie detective Endeavor Morse faces his firs...,Endeavour,False,6.60,19.00
45,46,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2009-01-27,0.00,38.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Hulk vs. Wolverine,False,6.80,48.00
46,46,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1997-01-19,0.00,0.00,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Last Stand at Saber River,False,3.70,3.00
47,46,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1995-01-01,0.00,0.00,[],Released,NaN,Running Wild,False,10.00,1.00


In [112]:
# DECISION & DOCUMENT: Drop Wiki data, as Kaggle data only has 3 NaN values

In [113]:
# Drop NaN values from Kaggle Dataset
movies_df = movies_df.dropna(subset=['release_date_kaggle'])


In [209]:
movies_df[movies_df['release_date_kaggle'].isnull() == True]

,Unnamed: 0,url,year,imdb_link,Based on,Starring,Cinematography,Running time,Country,Language,...,release_date_kaggle,revenue,runtime,spoken_languages,status,tagline,title_kaggle,video,vote_average,vote_count


In [213]:
# Language data

# Check wiki data for language categories 
movies_df['Language'].value_counts()

English                      5482
['English', 'Spanish']         68
['English', 'French']          35
['English', 'Japanese']        25
['English', 'Russian']         23
                             ... 
['English', 'Aramaic']          1
['Portuguese', 'English']       1
['English', 'Navajo']           1
['English', 'Ku']               1
['English', 'Hindi']            1
Name: Language, Length: 197, dtype: int64

In [214]:
# Check Kaggle data for langauge categories
movies_df['original_language'].value_counts(dropna=False)


en    6005
fr      17
es      10
it       8
de       6
ja       5
pt       4
zh       4
da       2
hi       2
pl       2
sv       1
ar       1
he       1
ab       1
ko       1
cn       1
ru       1
tr       1
Name: original_language, dtype: int64

In [ ]:
# Comparing the two datasets, the Wikipedia data is less consistent and will prove more costly to clean
# DECISION & DOCUMENT: Drop Wikipedia

In [215]:
# Production companies

movies_df[['Production company(s)','production_companies']]


,Production company(s),production_companies
0,Silver Pictures,[{'name': 'Twentieth Century Fox Film Corporat...
1,Avenue Pictures,"[{'name': 'Avenue Pictures Productions', 'id':..."
2,"['Carolco Pictures', 'IndieProd Company']","[{'name': 'IndieProd Company Productions', 'id..."
3,NaN,"[{'name': 'Orion Pictures', 'id': 41}]"
4,NaN,"[{'name': 'Paramount Pictures', 'id': 4}]"
...,...,...
6072,"['Fabula', 'Komplizen Film']","[{'name': 'Komplizen Film', 'id': 1618}, {'nam..."
6073,Ball & Chain Productions,"[{'name': 'Ball & Chain Productions', 'id': 74..."
6074,"['Arte France Cinéma', 'Why Not Productions']","[{'name': 'ARTE France Cinéma', 'id': 94}, {'n..."
6075,"['Film Science', 'Rough House Pictures', 'Sync...","[{'name': 'Film Science', 'id': 1976}, {'name'..."


In [ ]:
# Wiki                     Movielens                Resolution
#--------------------------------------------------------------------------
# title_wiki               title_kaggle             Drop Wiki
# running_time             runtime                  Keep Kaggle, replace 0 values with Wiki
# budget_wiki              budget_kaggle            Keep Kaggle, replace 0 values with Wiki
# box_office               revenue                  Keep Kaggle, replace 0 values with Wiki
# release_date_wiki        release_date_kaggle      Drop Wiki
# Language                 original_language        Drop Wiki
# Production company(s)    production_companies     Drop Wiki

In [217]:
# Execution



movies_df.drop(columns=['release_date_wiki','Language','Production company(s)'], inplace=True)


In [218]:
# Create a column to drop redundant columns in the future and replace missing Kaggle Data with wiki
def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
    df[kaggle_column] = df.apply(
        lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column]
        , axis=1)
    df.drop(columns=wiki_column, inplace=True)


In [220]:
# Use our function to apply our strategy to runtime, budget, and revenue columns
fill_missing_kaggle_data(movies_df, 'runtime', 'running_time')
fill_missing_kaggle_data(movies_df, 'budget_kaggle', 'budget_wiki')
fill_missing_kaggle_data(movies_df, 'revenue', 'box_office')
movies_df

KeyError: "['running_time'] not found in axis"

In [223]:
# Since we just merged data and done some transforms (filling in missing values), we should check if there are any columns with only one value (ie all data False in a Boolean column)
for col in movies_df.columns:
    value_counts = movies_df[col].value_counts(dropna=False)
    num_values = len(value_counts)
    if num_values == 1:
        print(col)

video


In [224]:
# Alternative list comprehension of the for loop above
[print(col) for col in movies_df.columns if len(movies_df[col].value_counts(dropna=False)) == 1]

video


[None]

In [227]:
# Drop the video column
movies_df.drop(columns='video',inplace=True)

In [230]:
# Reorder columns to present the dataframe more clearly:
# Consider the following organizational paradigm:
'''
Identifying information (IDs, titles, URLs, etc.)
Quantitative facts (runtime, budget, revenue, etc.)
Qualitative facts (genres, languages, country, etc.)
Business data (production companies, distributors, etc.)
People (producers, director, cast, writers, etc.)
'''
# Use the .loc() method to capture all rows and reorder the columns accordingly by passing in a list of reodrered column names
movies_df = movies_df.loc[:, ['imdb_id','id','title_kaggle','original_title','tagline','belongs_to_collection','url','imdb_link',
                       'runtime','budget_kaggle','revenue','release_date_kaggle','popularity','vote_average','vote_count',
                       'genres','original_language','overview','spoken_languages','Country',
                       'production_companies','production_countries','Distributor',
                       'Producer(s)','Director','Starring','Cinematography','Editor(s)','Writer(s)','Composer(s)','Based on'
                      ]]

# Alternatively we can pass in a new list into the index of the dataframe;
    # movies_df[['new_column_name_1', 'new_column_name_2']]

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['id', 'title_kaggle', 'url', 'budget_kaggle', 'release_date_kaggle',\n       ...\n       'Cinematography', 'Editor(s)', 'Writer(s)', 'Composer(s)', 'Based on'],\n      dtype='object', length=15). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [231]:
# Use the rename() method to pass in a dictionary or function, renaming values in the following format:
# {<id in column header>:<new id>}
# Docs: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html
movies_df.rename({'id':'kaggle_id',
                  'title_kaggle':'title',
                  'url':'wikipedia_url',
                  'budget_kaggle':'budget',
                  'release_date_kaggle':'release_date',
                  'Country':'country',
                  'Distributor':'distributor',
                  'Producer(s)':'producers',
                  'Director':'director',
                  'Starring':'starring',
                  'Cinematography':'cinematography',
                  'Editor(s)':'editors',
                  'Writer(s)':'writers',
                  'Composer(s)':'composers',
                  'Based on':'based_on'
                 }, axis='columns', inplace=True)

In [ ]:
# Because the ratings data is too lengthy, it will be difficult to cull succinct and actionable information from it rapidly. We can opt to use the groupby() function to quickly generate descriptive statistics for the concept of ("A movie is more popular if it receives more votes")

In [234]:

# Execute our groupby and rename the userId column to count
# Note the backslash allows us to chain a method but on a separate line for readability
rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
                .rename({'userId':'count'}, axis=1)

In [235]:
rating_counts

,movieId,rating,count,timestamp
0,1,0.50,441,441
1,1,1.00,804,804
2,1,1.50,438,438
3,1,2.00,2083,2083
4,1,2.50,1584,1584
...,...,...,...,...
239376,176267,4.00,1,1
239377,176269,3.50,1,1
239378,176271,5.00,1,1
239379,176273,1.00,1,1


In [236]:
# Use the pivot() method to pass in a new index, columns and values argument based off the original dataframe
rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
                .rename({'userId':'count'}, axis=1) \
                .pivot(index='movieId',columns='rating', values='count')

In [237]:
rating_counts

rating,0.50,1.00,1.50,2.00,2.50,3.00,3.50,4.00,4.50,5.00
movieId,,,,,,,,,,
1,441.00,804.00,438.00,"2,083.00","1,584.00","11,577.00","5,741.00","22,020.00","5,325.00","15,995.00"
2,263.00,797.00,525.00,"2,479.00","1,810.00","8,510.00","2,916.00","6,035.00",690.00,"2,035.00"
3,169.00,772.00,233.00,"1,665.00",616.00,"6,213.00",759.00,"3,433.00",154.00,"1,483.00"
4,47.00,351.00,31.00,496.00,77.00,"1,133.00",66.00,557.00,12.00,211.00
5,237.00,785.00,270.00,"1,716.00",664.00,"6,608.00",723.00,"2,959.00",166.00,"1,130.00"
...,...,...,...,...,...,...,...,...,...,...
176267,nan,nan,nan,nan,nan,nan,nan,1.00,nan,nan
176269,nan,nan,nan,nan,nan,nan,1.00,nan,nan,nan
176271,nan,nan,nan,nan,nan,nan,nan,nan,nan,1.00


In [238]:
# Rename the columns of the rating_counts dataframe for +readability
rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]

In [239]:
# Now that our rating_counts dataframe has the movieID as the index (aka kaggle_id), it acts as a PRIMARY KEY for a LEFT JOIN onto the movies_df dataframe

movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on='kaggle_id', right_index=True, how='left')


In [240]:
# Fill columns which return nan for ratings (aka they never received a rating in that specific category) with 0

movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0)


In [241]:
movies_with_ratings_df

,imdb_id,kaggle_id,title,original_title,tagline,belongs_to_collection,wikipedia_url,imdb_link,runtime,budget,...,rating_0.5,rating_1.0,rating_1.5,rating_2.0,rating_2.5,rating_3.0,rating_3.5,rating_4.0,rating_4.5,rating_5.0
0,tt0098987,9548,The Adventures of Ford Fairlane,The Adventures of Ford Fairlane,Kojak. Columbo. Dirty Harry. Wimps.,NaN,https://en.wikipedia.org/wiki/The_Adventures_o...,https://www.imdb.com/title/tt0098987/,104.00,"49,000,000.00",...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,tt0098994,25501,"After Dark, My Sweet","After Dark, My Sweet",All they risked was everything.,NaN,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",https://www.imdb.com/title/tt0098994/,114.00,"6,000,000.00",...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,tt0099005,11856,Air America,Air America,The few. The proud. The totally insane.,NaN,https://en.wikipedia.org/wiki/Air_America_(film),https://www.imdb.com/title/tt0099005/,112.00,"35,000,000.00",...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,tt0099012,8217,Alice,Alice,NaN,NaN,https://en.wikipedia.org/wiki/Alice_(1990_film),https://www.imdb.com/title/tt0099012/,102.00,"12,000,000.00",...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,tt0099018,25943,Almost an Angel,Almost an Angel,Who does he think he is?,NaN,https://en.wikipedia.org/wiki/Almost_an_Angel,https://www.imdb.com/title/tt0099018/,95.00,"25,000,000.00",...,3.00,0.00,3.00,2.00,5.00,26.00,37.00,46.00,16.00,11.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6072,tt5639354,429191,A Fantastic Woman,Una mujer fantástica,NaN,NaN,https://en.wikipedia.org/wiki/A_Fantastic_Woman,https://www.imdb.com/title/tt5639354/,104.00,nan,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6073,tt5390066,390059,Permission,Permission,NaN,NaN,https://en.wikipedia.org/wiki/Permission_(film),https://www.imdb.com/title/tt5390066/,96.00,nan,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6074,tt6304162,429174,Loveless,Нелюбовь,NaN,NaN,https://en.wikipedia.org/wiki/Loveless_(film),https://www.imdb.com/title/tt6304162/,128.00,nan,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6075,tt5795086,412302,Gemini,Gemini,NaN,NaN,https://en.wikipedia.org/wiki/Gemini_(2017_film),https://www.imdb.com/title/tt5795086/,92.00,nan,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
